In [30]:
# from model import *
from torchsummary import summary
from Uformer_model import *

In [32]:
# model = UNet_1(n_channels=18, n_classes=1)
model = Uformer(img_size=64,embed_dim=32,win_size=8,in_chans=1,dd_in=18,token_projection='linear',token_mlp='leff',modulator=False)

In [22]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [33]:
count_parameters(model)

20625851